In [1]:
# SQLite to BigQuery Export
# ========================
#
# This notebook converts a SQLite database (racing_data.db) to formats compatible with
# Google BigQuery, specifically CSV or JSON for direct upload.

import sqlite3
import pandas as pd
import os
import json
from tqdm.notebook import tqdm
import datetime
import gc  # For garbage collection
import time

# Set up directory for exports
EXPORT_DIR = "bigquery_exports"
os.makedirs(EXPORT_DIR, exist_ok=True)

# Database connection parameters
DB_PATH = 'racing_data.db'  # Update this if your database file is located elsewhere

# Maximum rows per file for chunking
# BigQuery has a 1GB limit per file, but we'll use a conservative number to ensure no issues
MAX_ROWS_PER_FILE = 500000

print(f"SQLite to BigQuery export tool initialized with export directory: {EXPORT_DIR}")


# Function to get a list of all tables in the database
def get_tables(db_path):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Query to get table names
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = [table[0] for table in cursor.fetchall()]
        
        conn.close()
        return tables
    except Exception as e:
        print(f"Error getting tables: {e}")
        return []


# Function to get a list of columns for a table
def get_column_info(db_path, table_name):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Query to get column information
        cursor.execute(f"PRAGMA table_info({table_name});")
        column_info = cursor.fetchall()
        
        # Extract column names and types
        columns = [{'name': col[1], 'type': col[2]} for col in column_info]
        
        conn.close()
        return columns
    except Exception as e:
        print(f"Error getting column info for table {table_name}: {e}")
        return []


# Function to export table to CSV format (BigQuery compatible)
def export_table_to_csv(db_path, table_name, export_dir, max_rows_per_file=500000):
    """
    Export a table to CSV format, chunking if necessary
    
    Args:
        db_path: Path to SQLite database
        table_name: Name of table to export
        export_dir: Directory to save exported files
        max_rows_per_file: Maximum rows per exported file
    
    Returns:
        list: Paths to exported files
    """
    try:
        conn = sqlite3.connect(db_path)
        
        # Get total row count
        row_count = pd.read_sql(f"SELECT COUNT(*) FROM {table_name}", conn).iloc[0, 0]
        print(f"Exporting table '{table_name}' with {row_count} rows")
        
        # Calculate number of chunks needed
        num_chunks = (row_count // max_rows_per_file) + (1 if row_count % max_rows_per_file > 0 else 0)
        
        exported_files = []
        
        for chunk_idx in tqdm(range(num_chunks), desc=f"Exporting {table_name}"):
            offset = chunk_idx * max_rows_per_file
            
            # Read chunk of data
            query = f"SELECT * FROM {table_name} LIMIT {max_rows_per_file} OFFSET {offset}"
            df_chunk = pd.read_sql(query, conn)
            
            # Handle any BigQuery incompatible data types
            for col in df_chunk.columns:
                # Convert any complex objects to strings
                if df_chunk[col].dtype == 'object':
                    df_chunk[col] = df_chunk[col].apply(lambda x: str(x) if x is not None else None)
            
            # Create filename with chunk information if needed
            file_suffix = f"_part{chunk_idx+1}of{num_chunks}" if num_chunks > 1 else ""
            filename = f"{table_name}{file_suffix}.csv"
            filepath = os.path.join(export_dir, filename)
            
            # Export to CSV
            df_chunk.to_csv(filepath, index=False)
            exported_files.append(filepath)
            
            # Force garbage collection
            del df_chunk
            gc.collect()
        
        conn.close()
        print(f"Exported {table_name} to {len(exported_files)} CSV file(s)")
        return exported_files
    
    except Exception as e:
        print(f"Error exporting table {table_name}: {e}")
        return []


# Function to export table to JSON format (BigQuery compatible)
def export_table_to_json(db_path, table_name, export_dir, max_rows_per_file=500000):
    """
    Export a table to JSON format (newline-delimited JSON), chunking if necessary
    
    Args:
        db_path: Path to SQLite database
        table_name: Name of table to export
        export_dir: Directory to save exported files
        max_rows_per_file: Maximum rows per exported file
    
    Returns:
        list: Paths to exported files
    """
    try:
        conn = sqlite3.connect(db_path)
        
        # Get total row count
        row_count = pd.read_sql(f"SELECT COUNT(*) FROM {table_name}", conn).iloc[0, 0]
        print(f"Exporting table '{table_name}' with {row_count} rows")
        
        # Calculate number of chunks needed
        num_chunks = (row_count // max_rows_per_file) + (1 if row_count % max_rows_per_file > 0 else 0)
        
        exported_files = []
        
        for chunk_idx in tqdm(range(num_chunks), desc=f"Exporting {table_name}"):
            offset = chunk_idx * max_rows_per_file
            
            # Read chunk of data
            query = f"SELECT * FROM {table_name} LIMIT {max_rows_per_file} OFFSET {offset}"
            df_chunk = pd.read_sql(query, conn)
            
            # Create filename with chunk information if needed
            file_suffix = f"_part{chunk_idx+1}of{num_chunks}" if num_chunks > 1 else ""
            filename = f"{table_name}{file_suffix}.json"
            filepath = os.path.join(export_dir, filename)
            
            # Convert to newline-delimited JSON (for BigQuery compatibility)
            # This creates one JSON object per line, which is what BigQuery expects
            with open(filepath, 'w') as f:
                f.write(df_chunk.to_json(orient='records', lines=True))
            
            exported_files.append(filepath)
            
            # Force garbage collection
            del df_chunk
            gc.collect()
        
        conn.close()
        print(f"Exported {table_name} to {len(exported_files)} JSON file(s)")
        return exported_files
    
    except Exception as e:
        print(f"Error exporting table {table_name}: {e}")
        return []


# Function to generate BigQuery schema from SQLite table
def generate_bigquery_schema(db_path, table_name):
    """
    Generate BigQuery schema JSON from SQLite table structure
    
    Args:
        db_path: Path to SQLite database
        table_name: Name of table
        
    Returns:
        list: BigQuery schema definition
    """
    column_info = get_column_info(db_path, table_name)
    
    # Map SQLite types to BigQuery types
    type_mapping = {
        'INTEGER': 'INTEGER',
        'INT': 'INTEGER',
        'SMALLINT': 'INTEGER',
        'TINYINT': 'INTEGER',
        'BIGINT': 'INTEGER',
        'REAL': 'FLOAT',
        'FLOAT': 'FLOAT',
        'DOUBLE': 'FLOAT',
        'TEXT': 'STRING',
        'VARCHAR': 'STRING',
        'CHAR': 'STRING',
        'BLOB': 'BYTES',
        'BOOLEAN': 'BOOLEAN',
        'DATE': 'DATE',
        'DATETIME': 'DATETIME',
        'TIMESTAMP': 'TIMESTAMP'
    }
    
    # Create schema
    schema = []
    for col in column_info:
        # Default to STRING for unknown types
        sqlite_type = col['type'].upper()
        bq_type = 'STRING'
        
        # Check for type matches
        for sql_type, big_query_type in type_mapping.items():
            if sqlite_type.startswith(sql_type):
                bq_type = big_query_type
                break
        
        schema.append({
            'name': col['name'],
            'type': bq_type,
            'mode': 'NULLABLE'
        })
    
    return schema


# Function to export all tables
def export_all_tables(db_path, export_dir, file_format='csv', max_rows_per_file=500000):
    """
    Export all tables from SQLite database to specified format
    
    Args:
        db_path: Path to SQLite database
        export_dir: Directory to save exports
        file_format: Format to export ('csv' or 'json')
        max_rows_per_file: Maximum rows per file
        
    Returns:
        dict: Information about exported tables
    """
    tables = get_tables(db_path)
    
    if not tables:
        print("No tables found in the database")
        return {}
    
    print(f"Found {len(tables)} tables: {', '.join(tables)}")
    
    export_info = {}
    start_time = time.time()
    
    for table in tables:
        table_start = time.time()
        
        # Create schema file for BigQuery
        schema = generate_bigquery_schema(db_path, table)
        schema_path = os.path.join(export_dir, f"{table}_schema.json")
        
        with open(schema_path, 'w') as f:
            json.dump(schema, f, indent=2)
        
        # Export table data
        if file_format.lower() == 'csv':
            exported_files = export_table_to_csv(db_path, table, export_dir, max_rows_per_file)
        else:  # json
            exported_files = export_table_to_json(db_path, table, export_dir, max_rows_per_file)
        
        table_time = time.time() - table_start
        
        export_info[table] = {
            'format': file_format,
            'files': exported_files,
            'schema_file': schema_path,
            'file_count': len(exported_files),
            'processing_time': f"{table_time:.2f} seconds"
        }
        
        print(f"Completed export of '{table}' in {table_time:.2f} seconds")
        
    total_time = time.time() - start_time
    print(f"Export completed in {total_time:.2f} seconds")
    
    # Write export summary
    summary_path = os.path.join(export_dir, 'export_summary.json')
    with open(summary_path, 'w') as f:
        summary = {
            'export_date': datetime.datetime.now().isoformat(),
            'database': db_path,
            'format': file_format,
            'tables': list(export_info.keys()),
            'total_processing_time': f"{total_time:.2f} seconds",
            'table_details': export_info
        }
        json.dump(summary, f, indent=2)
    
    return export_info


# Check if database file exists
if not os.path.exists(DB_PATH):
    print(f"ERROR: Database file '{DB_PATH}' not found. Please update the DB_PATH variable.")
else:
    print(f"Database file '{DB_PATH}' found.")
    # Tables in the database
    tables = get_tables(DB_PATH)
    print(f"Tables in database: {', '.join(tables)}")

# Run this cell to export all tables to CSV format (BigQuery compatible)
export_all_tables(DB_PATH, EXPORT_DIR, file_format='csv', max_rows_per_file=MAX_ROWS_PER_FILE)

# Alternatively, run this cell to export all tables to JSON format (BigQuery compatible)
# export_all_tables(DB_PATH, EXPORT_DIR, file_format='json', max_rows_per_file=MAX_ROWS_PER_FILE)

# Check the output files
files = os.listdir(EXPORT_DIR)
print(f"Files in export directory ({len(files)} files):")
for i, file in enumerate(sorted(files)):
    file_path = os.path.join(EXPORT_DIR, file)
    file_size = os.path.getsize(file_path) / (1024 * 1024)  # Size in MB
    print(f"{i+1}. {file} ({file_size:.2f} MB)")

SQLite to BigQuery export tool initialized with export directory: bigquery_exports
Database file 'racing_data.db' found.
Tables in database: races, runners, sqlite_sequence, sqlite_stat1
Found 4 tables: races, runners, sqlite_sequence, sqlite_stat1
Exporting table 'races' with 262679 rows


Exporting races:   0%|          | 0/1 [00:00<?, ?it/s]

Exported races to 1 CSV file(s)
Completed export of 'races' in 5.44 seconds
Exporting table 'runners' with 2543909 rows


Exporting runners:   0%|          | 0/6 [00:00<?, ?it/s]

Exported runners to 6 CSV file(s)
Completed export of 'runners' in 50.33 seconds
Exporting table 'sqlite_sequence' with 1 rows


Exporting sqlite_sequence:   0%|          | 0/1 [00:00<?, ?it/s]

Exported sqlite_sequence to 1 CSV file(s)
Completed export of 'sqlite_sequence' in 0.02 seconds
Exporting table 'sqlite_stat1' with 6 rows


Exporting sqlite_stat1:   0%|          | 0/1 [00:00<?, ?it/s]

Exported sqlite_stat1 to 1 CSV file(s)
Completed export of 'sqlite_stat1' in 0.02 seconds
Export completed in 55.82 seconds
Files in export directory (14 files):
1. export_summary.json (0.00 MB)
2. races.csv (75.12 MB)
3. races_schema.json (0.00 MB)
4. runners_part1of6.csv (188.29 MB)
5. runners_part2of6.csv (192.17 MB)
6. runners_part3of6.csv (199.84 MB)
7. runners_part4of6.csv (203.08 MB)
8. runners_part5of6.csv (204.74 MB)
9. runners_part6of6.csv (17.59 MB)
10. runners_schema.json (0.00 MB)
11. sqlite_sequence.csv (0.00 MB)
12. sqlite_sequence_schema.json (0.00 MB)
13. sqlite_stat1.csv (0.00 MB)
14. sqlite_stat1_schema.json (0.00 MB)
